In [75]:
import pandas as pd
import glob
import numpy as np 

In [52]:
glob.glob("tweet-sentiment-extraction/*.csv")

['tweet-sentiment-extraction/test.csv',
 'tweet-sentiment-extraction/train.csv',
 'tweet-sentiment-extraction/sample_submission.csv']

In [99]:
train_csv = pd.read_csv("tweet-sentiment-extraction/train.csv", index_col="textID")
test_csv = pd.read_csv("tweet-sentiment-extraction/test.csv", index_col="textID")
submit = pd.read_csv("tweet-sentiment-extraction/sample_submission.csv")

In [15]:
train_csv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27486 entries, a3d0a7d5ad to 1c1f3724db
Data columns (total 3 columns):
text             27485 non-null object
selected_text    27485 non-null object
sentiment        27486 non-null object
dtypes: object(3)
memory usage: 858.9+ KB


In [104]:
test_csv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3535 entries, 11aa4945ff to aa1a163174
Data columns (total 2 columns):
text         3535 non-null object
sentiment    3535 non-null object
dtypes: object(2)
memory usage: 82.9+ KB


In [68]:
# remove the null item in the train file
train_csv = train_csv[~((train_csv['text'].isnull()) | (train_csv['selected_text'].isnull()))]

In [9]:
import torch
from transformers import BertTokenizer

In [11]:
PRETRAINED_MODEL_NAME = "bert-large-cased"
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)


In [36]:
vocab = tokenizer.vocab
print("字典大小：", len(vocab))

字典大小： 28996


In [45]:
token = tokenizer.tokenize("I have an apple")
print("Convert token to id")
print(f"{token} ---> {tokenizer.convert_tokens_to_ids(token)}")

Convert token to id
['I', 'have', 'an', 'apple'] ---> [146, 1138, 1126, 12075]


In [88]:
# selecting the length of the text is less than 100 or equal to 100
MAX_LENGTH = 100
train_csv = train_csv[train_csv["text"].apply(lambda x: len(x)) <= MAX_LENGTH]
train_csv = train_csv[train_csv["selected_text"].apply(lambda x: len(x)) <= MAX_LENGTH]
print(f"train data remains after selecting: {len(train_csv)}")

train data remains after selecting: 21139


In [106]:
train_csv.columns = ["text1", "text2", "label"]
# idempotence, 將處理結果另存成 tsv 供 PyTorch 使用
train_csv.to_csv("train.tsv", sep="\t", index=False)

In [120]:
train_csv["label"].value_counts()

neutral     11118
positive     8582
negative     7786
Name: label, dtype: int64

In [ ]:
#  jaccard 